In [ ]:
# Importing all the Libraries and using Delivered Cosine_Similarty Function 
import re 
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise


In [3]:
# Reading Training DataSet 

trainDf = pd.read_csv(
    filepath_or_buffer='./train.dat', 
    header=None, 
    sep='\t')

print trainDf.head(5)

   0                                                  1
0 -1  Although a film with Bruce Willis is always wo...
1 -1  This movie was slower then Molasses in January...
2 -1  Interesting film about an actual event that to...
3 -1  It's painfully obvious that the people who mad...
4  1  This movie really is a mixed bag. On the one h...


In [4]:
# Reading Test DataSet 

testDf = pd.read_csv(
    filepath_or_buffer='./test.dat', 
    header=None, 
    sep='\t')

print testDf.head(5)

                                                   0
0  I am so glad when i watch in every time the mo...
1  when I first heard about this movie, I noticed...
2  I saw this on television more years ago than I...
3  Peter Boyle was always a great actor and he pr...
4  Another entertaining Travolta dance flick! GRE...


In [5]:
# Removed All Html Tags and Special Character and Number Except Space  from Trainging and Test data set 


# TrainingSet
trainvalues = trainDf.iloc[:,:].values

#print trainvalues

trainRatings = []
trainReviews = []

for value in trainvalues:
    trainRatings.append(value[0])
    trainReviews.append(re.sub('[^A-z -]', '', re.sub('<[^>]*>','',value[1])).lower())
    

# TestSet 

testvalues = testDf.iloc[:,:].values
testReviews = []

for value in testvalues:
    testReviews.append(re.sub('[^A-z -]', '', re.sub('<[^>]*>','',value[0])).lower())
       


In [6]:
def filterLen(docs, minlen):
    return[ [t for t in d if len(t) >= minlen ] for d in docs ]

In [27]:
'''
Removing All words whose lenth is less than 4 as those words 
doesnot add any value to analysis it is just a noise for us .
'''

trainDocs = [l.split() for l in trainReviews]
testDocs = [l.split() for l in testReviews]

trainDocs1 = filterLen(trainDocs,5)
testDocs1 =  filterLen(testDocs ,5)

#Before Processing 

print len(trainReviews[0])
print len(testReviews[0])

#After Processing 

print len(trainDocs1[0])
print len(testDocs1[0])



1985
1549
225
156


In [28]:
combinedList = trainDocs1 + testDocs1

In [29]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [30]:
combinedMatrix = build_matrix(combinedList)
combinedMatrix.shape
trainMatrix = combinedMatrix[0:25000]
testMatrix = combinedMatrix[25000:50000]

In [31]:
def cosine_similarity_n_space(m1, m2, batch_size=10):
    assert m1.shape[1] == m2.shape[1]
    ret = np.ndarray((m1.shape[0], m2.shape[0]))
    for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
        start = row_i * batch_size
        end = min([(row_i + 1) * batch_size, m1.shape[0]])
        if end <= start:
            break # cause I'm too lazy to elegantly handle edge cases
        rows = m1[start: end]
        sim = cosine_similarity(rows, m2) # rows is O(1) size
        ret[start: end] = sim
    return ret



In [32]:
cosineSimilarityValue = cosine_similarity_n_space(testMatrix,trainMatrix)


In [36]:

f = open('./format.dat', 'w')
count = 0
testIndex = 0
for row in cosineSimilarityValue:

    
    #kneighbours = heapq.nlargest(5, row)
    k=71
    partitioned_row_byindex = np.argpartition(-row, k)  
    similar_index = partitioned_row_byindex[:k]

    
    ReviewTypeNegative = 0
    ReviewTypePositive = 0

    for index in similar_index:

        if trainvalues[index][0] == -1:
               ReviewTypeNegative+=1
        elif trainvalues[index][0] == 1:
               ReviewTypePositive+=1
            
    
    if ReviewTypeNegative > ReviewTypePositive:
        f.write('-1\n')
        count +=1
    else:
        f.write('+1\n')
        count +=1
        
print("count : ",count)
print("TestIndex" , testIndex)
print("--The End--")


('count : ', 0)
--The End--
